# Assignment 2

In [1]:
import numpy as np
import scipy as sp
import matplotlib as mp
import matplotlib.pyplot as plt

from scipy.integrate import quad
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Question 5

### (a)

We have a loop of radius $0.1m$ lying in the xy-plane with current of $I$ flowing.

To calculate the magnetic field produced by this loop we use the Biot-Savart Law:

$$\vec{B}(\vec{r}) = \frac{\mu_0 I}{4 \pi} \int \frac{\text{d}l \times \hat{r'}}{r' ^2}$$

$$\text{where} \left\{ \begin{array}{ll}
            \vec{r}  & \text{is the position vector of the point of evaluation} \\
            \vec{r'} & \text{is the displacement vector from a point in the current loop to the point of evaluation} \\
            l        & \text{is the length of the current loop} \\
            \end{array} \right.$$      
            

**Let:**

>Radius of the current loop: $R$

>Angle from the center of the current loop to an element: $\theta$ 

>Line segment of the current loop: $l$

>Locations in space: $\vec{r} = (x, y, z)$

**Then:**

$$\text{d}l = R \ \text{d} \theta \ \hat{\theta} = [-R \sin(\theta), \ R \cos(\theta), \ 0] \text{d}\theta$$

$$\vec{r'} = [x-R\cos(\theta), \ y-R\sin(\theta), \ z]$$

**Taking the cross product:**

$$\begin{align}
\text{d}l \times \vec{r'} &= zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-R\sin(\theta)(y-R\sin(\theta)) - R\cos(\theta)(x-R\cos(\theta))] \ \hat{z} \ \text{d}\theta\\
&= zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z} \ \text{d}\theta
\end{align}$$

**The term inside the intergral:**
$$\begin{align}
\frac{\text{d}l \times \vec{r'}}{r' ^2} &= \frac{zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z}}{\left\{[x-R\cos(\theta)]^2 + [y-R\sin(\theta)]^2 + z^2 \right\}^\frac{3}{2} }\\
\end{align}$$

**The magnetic field:**
$$\vec{B}(\vec{r}) = \frac{\mu_0 I}{4 \pi} \int \text{d}\theta \ \frac{zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z}}{\left\{[x-R\cos(\theta)]^2 + [y-R\sin(\theta)]^2 + z^2 \right\}^\frac{3}{2} }$$

**Make it dimensionless:**
$$\vec{B}(\vec{r}) \frac{4 \pi}{\mu_0 I}  = \int \text{d}\theta \ \frac{\frac{z}{R}\cos(\theta) \ \hat{x} + \frac{z}{R}\sin(\theta) \ \hat{y} - \left[\frac{y}{R}\sin(\theta) + \frac{x}{R}\cos(\theta) - 1\right] \hat{z}}{\left\{ \left[\frac{x}{R}-\cos(\theta)\right]^2 + \left[\frac{y}{R}-\sin(\theta)\right]^2 + \left(\frac{z}{R}\right)^2 \right\}^\frac{3}{2} }$$

**Turn the magnetic field into code:**

In [2]:
# Magnetic fields in x
def integrand_x(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    Bx = (z/R)*np.cos(theta) / r**3
    return Bx

@np.vectorize
def Bx(x, y, z, R=1):
    B_x = quad(lambda theta: integrand_x(x, y, z, theta, R), 0, 2*np.pi)
    return B_x[0]

# Magnetic fields in y
def integrand_y(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    By = (z/R)*np.sin(theta) / r**3
    return By

@np.vectorize
def By(x, y, z, R=1):
    B_y = quad(lambda theta: integrand_y(x, y, z, theta, R), 0, 2*np.pi)
    return B_y[0]

# Magnetic fields in z
def integrand_z(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    Bz = -( (y/R)*np.sin(theta) + (x/R)*np.cos(theta) - 1) / r**3
    return Bz

@np.vectorize
def Bz(x, y, z, R=1):
    B_z = quad(lambda theta: integrand_z(x, y, z, theta, R), 0, 2*np.pi)
    return B_z[0]

We will now create a meshgrid for the space of integration.

In [3]:
# Dimesions and resolution for the meshgrid 
r_max = 2
r_min = -2
res = 20

x = np.linspace(r_min, r_max, res)
y = 0
z = np.linspace(r_min, r_max, res)

GX, GZ = np.meshgrid(x, z)
GY = y

Find the magnetic fields

In [4]:
%%time
B_x = Bx(GX, GY, GZ, 1)
B_y = By(GX, GY, GZ, 1)
B_z = Bz(GX, GY, GZ, 1)

CPU times: user 1.47 s, sys: 12.2 ms, total: 1.48 s
Wall time: 1.48 s


In [5]:
np.shape(B_z)

(20, 20)

In [6]:
fig, ax = plt.subplots(figsize=(7, 5))

norm = (B_x**2 + B_z**2)**0.5

Graph = ax.pcolormesh(x, z, norm, cmap='inferno_r')
ax.streamplot(x, z, B_x, B_z, linewidth=0.75,
             density=1.5, arrowstyle='->', arrowsize=1)

# ax.quiver(x, z, B_x/norm, B_z/norm, linewidth=0.75, pivot='mid')
# Graph = ax.imshow(E[2], extent=(-2, 2, -2, 2), vmax=100, cmap='inferno_r')

ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_xlabel(r'Distance ($\hat{x}$) [dm]', fontsize=12)
ax.set_ylabel(r'Distance ($\hat{z}$) [dm]', fontsize=12)
ax.set_title('Magnetic field of a current loop in the xz-plane', fontsize=14)

cbar = fig.colorbar(Graph, shrink=1, aspect=20)
cbar.ax.set_ylabel(r'$\left| \vec{B} \right|$ $\left[\frac{4 \pi}{\mu_0 I}\right]$', 
                  rotation=0, fontsize=16, labelpad=40)

caption = r"Magnetic field of a current loop located in the xy-plane traveling counterclockwise."
fig.text(0, -0.05, "\n".join(wrap(caption, 100)), ha='left', fontsize=12, wrap=False)

plt.show()

<Figure size 504x360 with 2 Axes>

### (b)